In [2]:
import pygtfs
from bs4 import BeautifulSoup as bsoup
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point
import fiona
from math import *
import numpy as np
from datetime import datetime, timedelta
from urllib.request import urlopen
import re
import difflib
from zipfile import ZipFile
import os
import sys 
from sklearn import preprocessing
from matplotlib import pyplot as plt
import statsmodels.formula.api as sm
import math

In [146]:
housing_df = pd.read_csv("Melbourne_housing_FULL.csv")

In [208]:
crime_df= pd.read_excel('Data_tables_Criminal_Incidents_Visualisation_year_ending_December_2018.xlsx'
                        ,sheet_name='Table 07')

In [209]:
crime_df.head(5)

,Year ending December,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,407
1,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,26
2,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,618
3,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,25
4,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",182


In [210]:
crime_df.isna().sum()

Year ending December    0
Postcode                0
Suburb/Town Name        0
Offence Division        0
Offence Subdivision     0
Offence Subgroup        0
Incidents Recorded      0
dtype: int64

In [211]:
crime_df=crime_df.rename(columns={'Year ending December':'Year'})

crime_groupby=crime_df.groupby(['Year','Postcode'], as_index=False)["Incidents Recorded"].sum()

In [212]:
crime_groupby.head(5)

,Year,Postcode,Incidents Recorded
0,2009,3000,17615
1,2009,3002,871
2,2009,3003,429
3,2009,3006,1369
4,2009,3008,507


In [213]:

crime_with_division=crime_df.groupby(['Year','Postcode','Offence Division']
                                     , as_index=False)["Incidents Recorded"].sum()
crime_with_division.head(5)

,Year,Postcode,Offence Division,Incidents Recorded
0,2009,3000,A Crimes against the person,1750
1,2009,3000,B Property and deception offences,10708
2,2009,3000,C Drug offences,822
3,2009,3000,D Public order and security offences,2876
4,2009,3000,E Justice procedures offences,1310


In [204]:
Melbourne_housing_df = pd.read_csv("Melbourne_housing_FULL.csv")

In [206]:
Melbourne_housing_df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [81]:
crime_df.head(5)

,Year,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,407
1,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,26
2,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,618
3,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,25
4,2009,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",182


In [148]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [149]:
df= Melbourne_housing_df.drop(columns=['BuildingArea','YearBuilt','Bedroom2','Bathroom','Car','Landsize'])

In [150]:
df=df.dropna()

In [140]:
df.head(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Landsize,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,202.0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,156.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,134.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,94.0,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,120.0,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019.0
10,Abbotsford,129 Charles St,2,h,941000.0,S,Jellis,7/05/2016,2.5,3067.0,181.0,Yarra City Council,-37.8041,144.9953,Northern Metropolitan,4019.0
11,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,245.0,Yarra City Council,-37.8024,144.9993,Northern Metropolitan,4019.0
14,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,256.0,Yarra City Council,-37.8060,144.9954,Northern Metropolitan,4019.0
17,Abbotsford,6/241 Nicholson St,1,u,300000.0,S,Biggin,8/10/2016,2.5,3067.0,0.0,Yarra City Council,-37.8008,144.9973,Northern Metropolitan,4019.0
18,Abbotsford,10 Valiant St,2,h,1097000.0,S,Biggin,8/10/2016,2.5,3067.0,220.0,Yarra City Council,-37.8010,144.9989,Northern Metropolitan,4019.0


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20993 entries, 1 to 34856
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         20993 non-null  object 
 1   Address        20993 non-null  object 
 2   Rooms          20993 non-null  int64  
 3   Type           20993 non-null  object 
 4   Price          20993 non-null  int64  
 5   Method         20993 non-null  object 
 6   SellerG        20993 non-null  object 
 7   Date           20993 non-null  object 
 8   Distance       20993 non-null  float64
 9   Postcode       20993 non-null  int64  
 10  CouncilArea    20993 non-null  object 
 11  Lattitude      20993 non-null  float64
 12  Longtitude     20993 non-null  float64
 13  Regionname     20993 non-null  object 
 14  Propertycount  20993 non-null  int64  
dtypes: float64(3), int64(4), object(8)
memory usage: 2.6+ MB


In [154]:
df.Price=df.Price.astype(int)
df.Postcode=df.Postcode.astype(int)
df.Propertycount=df.Propertycount.astype(int)

In [156]:
df['Date']=pd.to_datetime(df['Date'],format= '%d/%m/%Y')
df['Year']=df['Date'].dt.year
df['Month']=df['Date'].dt.month
df['Weekday']=df['Date'].dt.weekday

In [157]:
df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Year,Month,Weekday
1,Abbotsford,85 Turner St,2,h,1480000,S,Biggin,2016-12-03,2.5,3067,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019,2016,12,5
2,Abbotsford,25 Bloomburg St,2,h,1035000,S,Biggin,2016-02-04,2.5,3067,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019,2016,2,3
4,Abbotsford,5 Charles St,3,h,1465000,SP,Biggin,2017-03-04,2.5,3067,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019,2017,3,5
5,Abbotsford,40 Federation La,3,h,850000,PI,Biggin,2017-03-04,2.5,3067,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019,2017,3,5
6,Abbotsford,55a Park St,4,h,1600000,VB,Nelson,2016-06-04,2.5,3067,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019,2016,6,5


In [161]:
a=crime_with_division[crime_with_division['Year']>=2016]

In [162]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9792 entries, 21956 to 31747
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year                9792 non-null   int64 
 1   Postcode            9792 non-null   int64 
 2   Offence Division    9792 non-null   object
 3   Incidents Recorded  9792 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 382.5+ KB


In [163]:
final_df = pd.merge(df,crime_groupby, on=['Year','Postcode'])

In [165]:
final_df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Year,Month,Weekday,Incidents Recorded
0,Abbotsford,85 Turner St,2,h,1480000,S,Biggin,2016-12-03,2.5,3067,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019,2016,12,5,994
1,Abbotsford,25 Bloomburg St,2,h,1035000,S,Biggin,2016-02-04,2.5,3067,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019,2016,2,3,994
2,Abbotsford,55a Park St,4,h,1600000,VB,Nelson,2016-06-04,2.5,3067,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019,2016,6,5,994
3,Abbotsford,129 Charles St,2,h,941000,S,Jellis,2016-05-07,2.5,3067,Yarra City Council,-37.8041,144.9953,Northern Metropolitan,4019,2016,5,5,994
4,Abbotsford,124 Yarra St,3,h,1876000,S,Nelson,2016-05-07,2.5,3067,Yarra City Council,-37.8024,144.9993,Northern Metropolitan,4019,2016,5,5,994
5,Abbotsford,98 Charles St,2,h,1636000,S,Nelson,2016-10-08,2.5,3067,Yarra City Council,-37.8060,144.9954,Northern Metropolitan,4019,2016,10,5,994
6,Abbotsford,6/241 Nicholson St,1,u,300000,S,Biggin,2016-10-08,2.5,3067,Yarra City Council,-37.8008,144.9973,Northern Metropolitan,4019,2016,10,5,994
7,Abbotsford,10 Valiant St,2,h,1097000,S,Biggin,2016-10-08,2.5,3067,Yarra City Council,-37.8010,144.9989,Northern Metropolitan,4019,2016,10,5,994
8,Abbotsford,411/8 Grosvenor St,2,u,700000,VB,Jellis,2016-11-12,2.5,3067,Yarra City Council,-37.8110,145.0067,Northern Metropolitan,4019,2016,11,5,994
9,Abbotsford,40 Nicholson St,3,h,1350000,VB,Nelson,2016-11-12,2.5,3067,Yarra City Council,-37.8085,144.9964,Northern Metropolitan,4019,2016,11,5,994


In [125]:
#Extract the gtfs file
zip_gtfs = ZipFile('gtfs.zip')
zip_gtfs.extractall()

In [126]:
sched = pygtfs.Schedule(":memory:")                
# append data to schedule object
pygtfs.append_feed(sched, "./1/google_transit.zip")
pygtfs.append_feed(sched, "./2/google_transit.zip")

Loading GTFS data for <class 'pygtfs.gtfs_entities.Agency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Stop'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Route'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Trip'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.StopTime'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Service'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ServiceException'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Fare'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FareRule'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.ShapePoint'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Frequency'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Transfer'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.FeedInfo'>:
Loading GTFS data for <class 'pygtfs.gtfs_entities.Translation'>:
1 record read for <class 'pygtfs.gtfs_entities.Agency'>.
109 records read for <class 'pygtfs.gtfs_entities.Stop'>.
188 rec

In [166]:
# get the query of trips in schedule
sched_trips=sched.trips_query
# get the query of services in schedule
sched_services=sched.services_query
# get the query of stops in schedule
sched_stops=sched.stops_query
# get the query of stop_times in schedule
sched_stop_time=sched.stop_times_query.all()

#stops={}
stops_id=[]
station_name=[]
station_lat_lon=[]

# for loop in the query of stops
for i in sched_stops.all():
    stop_id=i.stop_id
    stop_name=i.stop_name
    stop_lat=i.stop_lat
    stop_lon=i.stop_lon
    #stops[stop_id]={'stop_name':stop_name,'stop_lat':stop_lat,'stop_lon':stop_lon}
    lat_lon=[stop_lat,stop_lon]
    #append stop name to station_name list
    
    
    if stop_id not in stops_id:
        station_name.append(stop_name)
        #append lat and long to lat_lon list
        station_lat_lon.append(lat_lon)
        #append stop id to stop_id list
        stops_id.append(stop_id)

# translate the list of station_lat_lon to matrix
station_lat_lon_mat=np.matrix(station_lat_lon)       

In [196]:
stops_id_name={}

for i in sched_stops.all():
    stop_id=i.stop_id
    stop_name=i.stop_name
    stop_lat=i.stop_lat
    stop_lon=i.stop_lon
    #stops[stop_id]={'stop_name':stop_name,'stop_lat':stop_lat,'stop_lon':stop_lon}
    lat_lon=[stop_lat,stop_lon]
    #append stop name to station_name list
    stops_id_name[stop_id] = stop_name

In [199]:
stops_id_name['20043']

'Southern Cross Railway Station (Melbourne City)'

In [167]:
# get the query of services in schedule
sched_service=sched.services_query.all()
weekday_service_id=[]

#for loop in sched_service
for i in sched_service:
    service_id=i.service_id
    M=int(i.monday)
    T=int(i.tuesday)
    W=int(i.wednesday)
    Th=int(i.thursday)
    F=int(i.friday)
    
    #if M+T+W+Th+F >=1, append service_id to weekday_service_id
    if M+T+W+Th+F == 5 and service_id not in weekday_service_id:
        weekday_service_id.append(service_id)

In [168]:
weekday_trip=[]
#for loop in the query of services in schedule
for i in sched_trips.all():
    route_id=i.route_id
    service_id=i.service_id
    trip_id=i.trip_id
    #if service_id in weekday_service_id, append trip_id to weekday_trip
    if service_id in weekday_service_id:
        weekday_trip.append(trip_id)

In [169]:
southern_cross_stop_id=['20043','22180']
# set a dictionary to store the trip and stop station id
trip_stop={}

# for loop in the query of stop_time
for i in sched_stop_time:
    stop_list=[]
    stop_id=i.stop_id
    trip_id=i.trip_id
    # append stop_id to stop_list
    stop_list.append(stop_id)
    arrival_time=i.arrival_time
    departure_time=i.departure_time
    # if departure_time between '7:00:00' and '9:30:00', trip_id in trip_list, stop_id equals to station_id, append departure_time to departure_time_list
    if str(departure_time) >= '7:00:00' and str(departure_time) < '9:30:00' and trip_id in weekday_trip: 
        if trip_id not in trip_stop.keys():
            trip_stop[trip_id]=stop_list
        else:
            original_value=trip_stop[trip_id]
            original_value.append(stop_id)
            trip_stop[trip_id]=original_value
    # if trip_id in trip_stop.keys and if stop_id equals to southern_corss_stop_id, add the value to trip_stop[trip_id]
    elif trip_id in trip_stop.keys():
        if stop_id == southern_cross_stop_id[0]:
            original_value=trip_stop[trip_id]
            original_value.append(southern_cross_stop_id[0])
            trip_stop[trip_id]=original_value
        elif stop_id == southern_cross_stop_id[1]:
            original_value=trip_stop[trip_id]
            original_value.append(southern_cross_stop_id[1])
            trip_stop[trip_id]=original_value
            

In [170]:
def get_distance(target,house):
    # radius of equator: KM
    ra = 6378  
    
    # change angle to radians
    target_rad_lat = np.radians(target[:,0])
    target_rad_lon = np.radians(target[:,1])
    house_rad_lat = np.radians(house[:,0])
    house_rad_lon = np.radians(house[:,1])
    
    dlon = house_rad_lon - target_rad_lon
    dlat = house_rad_lat - target_rad_lat
    a=np.power((np.sin(dlat/2)),2)+ np.multiply(np.multiply(np.cos(target_rad_lat),np.cos(house_rad_lat)),np.power((np.sin(dlon/2.0)),2))
    c = 2 * np.arcsin(np.sqrt(a))
    distance = ra * c *1000
    
    return distance

In [176]:
def find_station_id(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between station.
    all_distance = get_distance(station_lat_lon_mat,Find_loc)
    #list 100 lowest distance
    rank_100=list(all_distance.argsort(axis=0)[:100])
    #for loop in rank_100
    for i in rank_100:
        # if station id equals to southern_cross_stop_id, return southern_cross_stop_id, and break the function
        if stops_id[int(i)] == southern_cross_stop_id[0]:
            return southern_cross_stop_id[0]
            break
        elif stops_id[int(i)] == southern_cross_stop_id[1]:
            return southern_cross_stop_id[1]
            break
            
        else:
            #for loop in the values in the dictionary of trip_stop 
            for j in trip_stop.values(): 
                # if target station id and southern_corss_station id all in trip, return station name and distance
                if stops_id[int(i)] in j and southern_cross_stop_id[0] in j and j.index(stops_id[int(i)]) < j.index(southern_cross_stop_id[0]):
                    station=stops_id[int(i)]
                    distance=round(float(all_distance[int(i)]),3)
                    return station
                    break
                elif stops_id[int(i)] in j and southern_cross_stop_id[1] in j and j.index(stops_id[int(i)]) < j.index(southern_cross_stop_id[1]):
                    station=stops_id[int(i)]
                    distance=round(float(all_distance[int(i)]),3)
                    return station
                    break

In [200]:
def find_station_name(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between station.
    all_distance = get_distance(station_lat_lon_mat,Find_loc)
    #list 100 lowest distance
    rank_100=list(all_distance.argsort(axis=0)[:100])
    #for loop in rank_100
    for i in rank_100:
        # if station id equals to southern_cross_stop_id, return southern_cross_stop_id, and break the function
        if stops_id[int(i)] == southern_cross_stop_id[0]:
            station_name='Southern Cross Railway Station'
            return station_name
            break
        elif stops_id[int(i)] == southern_cross_stop_id[1]:
            station_name='Southern Cross Railway Station'
            return station_name
            break 
            
        else:
            #for loop in the values in the dictionary of trip_stop 
            for j in trip_stop.values(): 
                # if target station id and southern_corss_station id all in trip, return station name and distance
                if stops_id[int(i)] in j and southern_cross_stop_id[0] in j and j.index(stops_id[int(i)]) < j.index(southern_cross_stop_id[0]):
                    station=stops_id[int(i)]
                    distance=round(float(all_distance[int(i)]),3)
                    station_name=stops_id_name[station]
                    return station_name
                    break
                elif stops_id[int(i)] in j and southern_cross_stop_id[1] in j and j.index(stops_id[int(i)]) < j.index(southern_cross_stop_id[1]):
                    station=stops_id[int(i)]
                    distance=round(float(all_distance[int(i)]),3)
                    station_name=stops_id_name[station]
                    return station_name
                    break

In [177]:
def find_station_distance(lat,lon):
    Find_loc = np.matrix([[lat,lon]])
    #run get_distance function to get all distance between station.
    all_distance = get_distance(station_lat_lon_mat,Find_loc)
    #list 500 lowest distance
    rank_10=list(all_distance.argsort(axis=0)[:100])
    for i in rank_10:
        for j in trip_stop.values(): 
            # if target station id and southern_corss_station id all in trip, return station name and distance
            if stops_id[int(i)] in j and (southern_cross_stop_id[0] in j or southern_cross_stop_id[1] in j):
                station=stops_id[int(i)]
                distance=round(float(all_distance[int(i)]),3)
                return distance
                break

In [182]:
def average_min(station_id):
    station_id=station_id
    if station_id not in southern_cross_stop_id:
        trip_list=[]
        for key,values in trip_stop.items():
            if station_id in values and southern_cross_stop_id[0] in values and values.index(station_id)<values.index(southern_cross_stop_id[0]):
                if key in weekday_trip:
                    trip_list.append(key)
            elif station_id in values and southern_cross_stop_id[1] in values and values.index(station_id)<values.index(southern_cross_stop_id[1]):
                if key in weekday_trip:
                    trip_list.append(key)   
        departure_time_list=[]
        arrival_time_list=[]    
        #for loop in sched_stop_time
        for i in sched_stop_time:
            stop_id=i.stop_id
            trip_id=i.trip_id
            arrival_time=i.arrival_time
            departure_time=i.departure_time
            # if trip_id in trip_list, stop_id equals to station_id, append departure_time to departure_time_list
            if trip_id in trip_list and stop_id == station_id: 
                departure_time_list.append(departure_time)
            #if trip_id in trip_list and stop_id equals to southern_cross_stop_id, append arrival_time to arrival_time_list
            if trip_id in trip_list and stop_id in southern_cross_stop_id: 
                arrival_time_list.append(arrival_time)            
        #calculate the average by minus arrival_time and departure_time
        sum_second=0
        count=0
        for i in range(len(arrival_time_list)):
            sum_second+=(arrival_time_list[i]-departure_time_list[i]).seconds
            count+=1
        average_min=(sum_second/60)/count    
    #if the station_id is equals to southern cross station, the average_min equals to zero
    else:
        average_min=0   
    #return average_min
    return average_min

In [178]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20993 entries, 0 to 20992
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Suburb                     20993 non-null  object        
 1   Address                    20993 non-null  object        
 2   Rooms                      20993 non-null  int64         
 3   Type                       20993 non-null  object        
 4   Price                      20993 non-null  int64         
 5   Method                     20993 non-null  object        
 6   SellerG                    20993 non-null  object        
 7   Date                       20993 non-null  datetime64[ns]
 8   Distance                   20993 non-null  float64       
 9   Postcode                   20993 non-null  int64         
 10  CouncilArea                20993 non-null  object        
 11  Lattitude                  20993 non-null  float64       
 12  Long

In [183]:
final_df['train_station_id']=final_df.apply(lambda x: find_station_id(x.Lattitude, x.Longtitude),axis=1)
final_df['distance_to_train_station']=final_df.apply(lambda x: find_station_distance(x.Lattitude, x.Longtitude),axis=1)
final_df['travel_min_to_CBD']=final_df.apply(lambda x: average_min(x.train_station_id),axis=1)
final_df['train_station_name']=final_df.apply(lambda x: find_station_name(x.Lattitude, x.Longtitude),axis=1)


In [214]:
final_df.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Regionname,Propertycount,Year,Month,Weekday,Incidents Recorded,train_station_id,distance_to_train_station,travel_min_to_CBD,train_station_name
0,Abbotsford,85 Turner St,2,h,1480000,S,Biggin,2016-12-03,2.5,3067,...,Northern Metropolitan,4019,2016,12,5,994,19975,350.781,13.235294,Victoria Park Railway Station (Abbotsford)
1,Abbotsford,25 Bloomburg St,2,h,1035000,S,Biggin,2016-02-04,2.5,3067,...,Northern Metropolitan,4019,2016,2,3,994,19977,289.148,10.125000,North Richmond Railway Station (Richmond)
2,Abbotsford,55a Park St,4,h,1600000,VB,Nelson,2016-06-04,2.5,3067,...,Northern Metropolitan,4019,2016,6,5,994,19976,299.261,12.235294,Collingwood Railway Station (Abbotsford)
3,Abbotsford,129 Charles St,2,h,941000,S,Jellis,2016-05-07,2.5,3067,...,Northern Metropolitan,4019,2016,5,5,994,19976,144.364,12.235294,Collingwood Railway Station (Abbotsford)
4,Abbotsford,124 Yarra St,3,h,1876000,S,Nelson,2016-05-07,2.5,3067,...,Northern Metropolitan,4019,2016,5,5,994,19976,542.509,12.235294,Collingwood Railway Station (Abbotsford)


In [203]:
final_df.to_csv("housing_formated.csv",index=False)